In [2]:
import json

import torch
import wandb

from src.data_processing.data_loader import data_loader
from src.models.Dir_VRNN.model import DirVRNN



In [8]:
import os, pathlib

os.path.exists("src/models/Dir_VRNN/run_config.json")

True

In [23]:
# Load Configuration 
with open("src/models/Dir_VRNN/run_config.json", "r") as f:
    run_config = json.load(f)
    f.close()

# Extract info
data_config = run_config["data_config"]
model_config = run_config["model_config"]
training_config = run_config["training_config"]

In [12]:

# Load and process data
data_info = data_loader(**data_config)

/home/ball4537/PycharmProjects/VarPhenClustering/src/data_processing/data_loading_utils.py:453: FutureWarning: is_monotonic is deprecated and will be removed in a future version. Use is_monotonic_increasing instead.
  cond1 = X[self.id_col].is_monotonic



MIMIC data has been subsettted to the following features: 
 ['DBP', 'ESI', 'HR', 'RR', 'SBP', 'SPO2', 'TEMP', 'age', 'gender'].


100%|██████████| 8328/8328 [00:10<00:00, 811.68it/s]



MIMIC data successfully loaded.

Basic information 
 Input shape: (8328, 10, 9), (8328, 4) 
 Outcome Distribution: Death          26
ICU          1321
Ward         4121
Discharge    2860
dtype: int64


In [13]:
# GPU and model setting
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(run_config["model_config"]["seed"])
torch.backends.cudnn.deterministic = True
print("Device is ", device)


Device is  cuda


In [43]:
from importlib import reload
import src.models.Dir_VRNN.model as BaseModels

reload(BaseModels)


model = BaseModels.DirVRNN(**model_config, device=device).to(device)
model.fit(data_info=data_info, train_info=training_config, run_config=run_config)
torch.save(model.state_dict(), "DirVRNN.h5")

Train epoch 1 (100%):  [L1339.05070 - loglik -443.37512 - kl 10.61951 - out 22.93312]
Predict test (100%):  [L74.09708 - loglik -25.09204 - kl 0.09792 - out 1.47280]
Train epoch 2 (100%):  [L991.15763 - loglik -343.54895 - kl -5.09739 - out 24.19709]
Predict test (100%):  [L40.92328 - loglik -15.05167 - kl -0.88431 - out 1.57879]
Train epoch 3 (100%):  [L233.04912 - loglik -118.08885 - kl -31.68364 - out 26.16650]
Predict test (100%):  [L-22.93550 - loglik 2.90338 - kl -4.16274 - out 1.73715]
Train epoch 4 (100%):  [L-1403.67696 - loglik 256.12103 - kl -201.67841 - out 30.27864]
Predict test (100%):  [L-172.18044 - loglik 27.09557 - kl -29.54186 - out 2.26814]
Train epoch 5 (100%):  [L-4472.83080 - loglik 554.07166 - kl -923.01257 - out 41.57863]
Predict test (100%):  [L-416.16492 - loglik 40.34589 - kl -97.34247 - out 3.09986]


In [36]:
training_config

{'lr': 0.001, 'epochs': 5, 'bs': 128}